In [27]:
import pandas as pd
import datasets
from torch.utils.data import DataLoader

# Define constants
msmarco_path = "/kaggle/input/ms-marco-hr/ms-marco-translated.csv"
wiki_path = "/kaggle/input/wiki-cro/wiki-cro.csv"

# Load datasets
msmarco_df = pd.read_csv(msmarco_path)
wiki_df = pd.read_csv(wiki_path)

msmarco_df = msmarco_df.rename(columns={"finalpassage_cro": "passage", "query_cro": "query", "query": "query_eng"})
wiki_df = wiki_df.rename(columns={"Query": "query", "Summary": "passage"})

# Select only the new columns
msmarco_df = msmarco_df[["query", "passage"]]
wiki_df = wiki_df[["query", "passage"]]

combined_df = pd.concat([msmarco_df, wiki_df], ignore_index=True)


In [28]:
# Convert to Hugging Face Dataset
ds = datasets.arrow_dataset.Dataset.from_pandas(combined_df)
ds = ds.filter(lambda example: example["passage"] is not None and example["query"] is not None)

# Split the dataset
seed = 42
_train_val, test_ds = ds.train_test_split(test_size=0.1, seed=seed).values()
train_ds, val_ds = _train_val.train_test_split(test_size=0.1111, seed=seed).values()

# Save splits to CSV
train_ds.to_csv("train.csv", index=False)
val_ds.to_csv("val.csv", index=False)
test_ds.to_csv("test.csv", index=False)

print(len(train_ds), len(val_ds), len(test_ds))

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

15879 1985 1985


In [29]:
# Load splits from CSV
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Create DataLoaders (if required)
train_dataloader = DataLoader(train_df, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_df, batch_size=32, shuffle=False)

print("Data pipeline completed successfully.")

Data pipeline completed successfully.
